In [ ]:
%%capture
!pip3 install bnbphoneticparser
# !pip3 install googletrans==4.0.0-rc1 
!pip3 uninstall googletrans
!pip3 install googletrans==3.1.0a0
!pip3 install lingua-language-detector
!apt update
!apt install enchant --fix-missing
!pip3 install pyenchant==2.0.0
!pip3 install emoji

In [ ]:
import googletrans
from bnbphoneticparser import BanglishToBengali
from bnbphoneticparser import BengaliToBanglish
import time
import string 
import re
import enchant
from lingua import Language, LanguageDetectorBuilder
import emoji
import pandas as pd

In [ ]:
# coding=utf-8
translator = googletrans.Translator()
banglish2bengali = BanglishToBengali()
bengali2banglish = BengaliToBanglish()
d = enchant.Dict("en_US")
languages = [Language.ENGLISH, Language.BENGALI]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
special_characters = "!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~।"

In [ ]:
def spaceAllocate(txt):
  txt = str(txt)
  text = txt.translate(str.maketrans({key: " {0} ".format(key) for key in special_characters}))
  return text

In [ ]:
def spliteKeyWord(str):
    regex = r"[\u0980-\u09FF]+|[0-9]+|[,.।]+|[a-zA-Z]+\'*[a-z]*"
    matches = re.findall(regex, str, re.UNICODE)
    return matches

In [ ]:
def processSplitting(lst):
  new_lst = []
  for wrd in lst:
    result = spliteKeyWord(wrd)
    if len(result) == 1:
      new_lst.append(result[0])
      continue
    else:
      for i in range(len(result)):
        new_lst.append(result[i])
  return new_lst

In [ ]:
def clean_txt(txt):
  txt = txt.replace("\n", " ")
  txt = emoji.replace_emoji(txt)
  bn_txt = re.split(r'[` \=~!#@$%^&.(),\—\_\-\[\]{};:\'\\"|<<>?]''', txt)
  bn_txt = list(filter(lambda item: item.strip(), bn_txt))
  return bn_txt

In [ ]:
def BFRD_Translate(txt):
  trns_txt = ""

  txt = spaceAllocate(txt)
  s_txt = clean_txt(txt)
  bn_txt = processSplitting(s_txt)

  for element in bn_txt:
    if element == "।":
      trns_txt = trns_txt + " " + element
      continue
    elif d.check(element):
      #print(element)
      if element.isdigit():
        trns_txt = trns_txt + " " + translator.translate(element, src='bn', dest='bn').text
      else:
        trns_txt = trns_txt + " " + translator.translate(element, src='en', dest='bn').text
    else:
      if element.isdigit():
        trns_txt = trns_txt + " " + translator.translate(element, src='bn', dest='bn').text
      elif detector.detect_language_of(element) == Language.BENGALI:
        trns_txt = trns_txt + " " + element
        continue
      else:
        if detector.detect_language_of(element) == Language.ENGLISH:
          trns_txt = trns_txt + " " + translator.translate(element, src='en', dest='bn').text
        else:
          trns_txt = trns_txt + " " + banglish2bengali.parse(element)
    time.sleep(1)

  return trns_txt

In [ ]:
def Second_level_Translate(txt):
    trns_txt = ""
    txt = spaceAllocate(txt)
    txt = clean_txt(txt)
    cn_txt = processSplitting(txt)
    #print(cn_txt)
    for element in cn_txt:
        if element == "।":
          trns_txt = trns_txt + " " + element
          continue
        else:
            if detector.detect_language_of(element) == Language.ENGLISH:
               trns_txt = trns_txt + " " + banglish2bengali.parse(element)
            else:
                trns_txt = trns_txt + " " + element
                continue
    return trns_txt

In [ ]:
df = pd.read_excel("/content/fake.xlsx")

In [ ]:
result_list = []

for txt in df.Review:
  x = BFRD_Translate(txt)
  y = Second_level_Translate(x)
  result_list.append(y)

In [ ]:
new_df = pd.DataFrame()
new_df['Translated'] = result_list

In [ ]:
new_df.to_csv('Translated_Fake.csv', encoding = 'utf-8-sig', index=False)